In [1]:
import data as data_lib
import tensorflow_datasets as tfds
import tensorflow as tf
import argparse

In [2]:
builder = tfds.builder('cifar10')
builder.download_and_prepare()

In [26]:
FLAGS = argparse.Namespace(
    image_size=32,color_jitter_strength=1.0,eval_split='test',cache_dataset=False,train_split='train',train_mode='finetune'
)

In [27]:
data_lib.FLAGS =FLAGS

In [28]:
ds = data_lib.build_distributed_dataset(builder,4,True,
                                           tf.distribute.MirroredStrategy(), None)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [4]:
ds

In [29]:
for batch in ds:
    print(batch[0].shape)
    print(batch[1].shape)
    break

(4, 32, 32, 3)
(4, 10)


2023-08-28 06:17:03.788794: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]
2023-08-28 06:17:03.789229: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-08-28 06:17:03.795071: W tensorflow/core/grappler/optimizers/data/slack.cc:103] Could not find a final `prefetch` in the input pipeline to which to introduce slack.
2023-08-28 06:17:03.848784: W tensorflow/core/framework/dataset.cc:807] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset

In [7]:
ds[0][0].shape

TypeError: 'DistributedDatasetsFromFunction' object is not subscriptable

In [2]:
import pickle
with open('spectrograms/entire_data.pickle','rb') as f:
      ds=pickle.load(f)

In [3]:
ds[0][0].shape

(4, 480, 640, 3)

In [10]:
import model as model_lib

In [15]:
import resnet

In [61]:
FLAGS = argparse.Namespace(
    resnet_depth=18,
    width_multiplier=1,
    image_size=32,
    train_mode='pretrain',
    fine_tune_after_block=-1,
    global_bn=True,
    batch_norm_decay=0.9,
    sk_ratio=0,se_ratio=0,
    proj_out_dim=128,proj_head_mode='nonlinear',
    num_proj_layers=3,
    lineareval_while_pretraining=True,use_blur=False,
    ft_proj_selector=0
)


In [62]:
model_lib.FLAGS =FLAGS

In [63]:
resnet.FLAGS =FLAGS

In [64]:
model_check= model_lib.Model(3)

In [65]:
projection_head_outputs, supervised_head_outputs = model_check(
            ds[0][0], training=True)

In [66]:
supervised_head_outputs

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[-0.02441762, -0.08470368, -0.01879977],
       [-0.01582483, -0.05063347, -0.01692815],
       [-0.01194347, -0.05021737, -0.0227459 ],
       [-0.01154371, -0.05502656, -0.02288017]], dtype=float32)>